# Label Argilla datasets with LLMs, good annotation guidelines, and distilabel

In this notebook, we will use the `distilabel` library to label datasets with LLMs. We will use the `ArgillaLabeller` class to label any dataset that is hosted on Argilla with based on the written dataset definitions, like guidelines, fields, and questions. This is cool because you can use the same labelling interface for any dataset and questions that is hosted on Argilla.

## Getting started

### Deploy the Argilla server¶

If you already have deployed Argilla, you can skip this step. Otherwise, you can quickly deploy Argilla following [this guide](https://docs.argilla.io/latest/getting_started/quickstart/).

### Install dependencies

In [12]:
!pip install "git+https://github.com/argilla-io/distilabel.git@develop#egg=distilabel[llama_cpp]"

DEPRECATION: git+https://github.com/argilla-io/distilabel.git@develop#egg=distilabel[llama_cpp] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/argilla-io/distilabel.git (to revision develop) to /private/var/folders/8z/jnnncfnj7_lfxym0262z4p180000gn/T/pip-install-d1dwfym7/distilabel_27023bb5907f435ca6caae42d5139507
  Running command git clone --filter=blob:none --quiet https://github.com/argilla-io/distilabel.git /private/var/folders/8z/jnnncfnj7_lfxym0262z4p180000gn/T/pip-install-d1dwfym7/distilabel_27023bb5907f435ca6caae42d5139507
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  branch 'develop' set up to track 'origin/develop'.
  Resolved https://github.com/argilla-io/distilabel.git to commit 303722c58834f

In [ ]:

!pip install -U -qqq "numpy==1.26.4" \
                     "outlines==0.0.36" \
                     "llama_cpp_python==0.2.85" \
                     "argilla==2.4.1"

### Download the model

For the example, we will just use a [quantized version of llama 3.2 1B](https://huggingface.co/hugging-quants/Llama-3.2-1B-Instruct-Q8_0-GGUF/tree/main). This model will be good enough for basic labelling tasks.

In [2]:
url = "https://huggingface.co/hugging-quants/Llama-3.2-1B-Instruct-Q8_0-GGUF/resolve/main/llama-3.2-1b-instruct-q8_0.gguf?download=true"
filename = "llama-3.2-1b-instruct-q8_0.gguf"

import requests
import os


def download_file(url, filename):
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(filename, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)


if not os.path.exists(filename):
    print(f"Downloading {filename}...")
    download_file(url, filename)
    print(f"Download complete: {filename}")
else:
    print(f"{filename} already exists. Skipping download.")

llama-3.2-1b-instruct-q8_0.gguf already exists. Skipping download.


## Upload some a basic datasets to Argilla

We will now choose 4 datasets from Hugging Face and upload them to Argilla. A dataset each for the `TextQuestion` and `LabelQuestion`.

- https://huggingface.co/datasets/dair-ai/emotion: emotions as classes  
- https://huggingface.co/datasets/fka/awesome-chatgpt-prompts: chatgpt prompts as text

We will use the `rg.Dataset.from_hub` method to upload example datasets to Argilla. We will also use the `rg.Settings` class to map the fields to the questions.

In [9]:
import os
from uuid import uuid4

import argilla as rg

client = rg.Argilla(api_key="argilla.apikey", api_url="http://localhost:6900")

settings = rg.Settings(
    fields=[
        rg.TextField(
            name="text",
            title="Text",
            description="Provide a concise response to the prompt",
        )
    ],
    questions=[
        rg.LabelQuestion(
            name="emotion",
            title="Emotion",
            description="Provide a single label for the emotion of the text",
            labels=["joy", "anger", "sadness", "fear", "surprise", "love"],
        )
    ],
    mapping={"text": "text"},
)

dataset_name = f"emotion-{uuid4()}"

rg.Dataset.from_hub(
    repo_id="dair-ai/emotion",
    name=dataset_name,
    split="train[:100]",
    client=client,
    with_records=True,
    settings=settings,
)

/Users/ben/code/argilla-cookbook/.venv/lib/python3.11/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: argilla id: 735cae0d-eb08-45c3-ad79-0a11ad4dd2c2
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")
/Users/ben/code/argilla-cookbook/.venv/lib/python3.11/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['label'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")


DatasetRecords: The provided batch size 256 was normalized. Using value 100.

Sending records...: 100%|██████████| 1/1 [00:00<00:00,  6.68batch/s]


Dataset(id=UUID('55bb1f50-710a-412b-a411-4dab8349d4c2') inserted_at=datetime.datetime(2024, 10, 9, 13, 42, 27, 624381) updated_at=datetime.datetime(2024, 10, 9, 13, 42, 27, 751505) name='emotion-d5588445-108b-4e73-b1b8-f984fbb6281a' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('735cae0d-eb08-45c3-ad79-0a11ad4dd2c2') last_activity_at=datetime.datetime(2024, 10, 9, 13, 42, 27, 751505))

## Label the datasets

We will now label the datasets. We will use the `ArgillaLabeller` class to label the datasets. This class will use  will use a `LlamaCppLLM` LLM to label the datasets. These labels will then be converted into `rg.Suggestion` objects and added to the records. For the sake of the example, we will only label 5 records per time using a while loop that continuesly fetches pending records from Argilla for both datasets and labels them with the LLM. After the labelling, we will update the dataset with the new records.

In [14]:
from distilabel.llms.llamacpp import LlamaCppLLM
from distilabel.steps.tasks import ArgillaLabeller


# Initialize the labeller with the model and fields
labeller = ArgillaLabeller(
    llm=LlamaCppLLM(
        model_path="llama-3.2-1b-instruct-q8_0.gguf",
        n_ctx=8000,
        extra_kwargs={"max_new_tokens": 8000, "temperature": 0.0},
    )
)
labeller.load()

dataset = client.datasets(name=dataset_name, workspace="argilla")
pending_records = list(
    dataset.records(
        query=rg.Query(filter=rg.Filter(("status", "==", "pending"))),
        limit=1,
    )
)

print(pending_records)

[Record(id=ead1145a-79e5-4170-aa27-d64584adff7e,status=pending,fields={'text': 'i didnt feel humiliated'},metadata={},suggestions={},responses={})]


## Distilabel will define a fewshot prompt for you

For the sake of understanding the labelling process, we will expose the prompt that is used to label the records. This prompt is generated by the `ArgillaLabeller` class and is based on the questions and fields of the dataset. This prompt is used to label the records with the LLM.

In [26]:
from rich import print

prompt = [
    labeller.format_input(
        {
            "record": record,
            "fields": dataset.fields,
            "question": dataset.questions[0],
            "guidelines": dataset.guidelines,
        }
    )
    for record in pending_records
][0]

print(*[row["content"] for row in prompt], sep="\n")

You are an expert annotator and labelling assistant that understands complex domains and natural language 
processing. You are given input fields and a question. You should create a valid JSON object as an answer to the 
question based on the input fields. 1. Understand the input fields and optional guidelines. 2. Understand the 
question type and the question settings. 3. Reason through your response step-by-step. 4. Provide a valid JSON 
object as an answer to the question.
Please provide an answer to the question based on the input fields.

# Input Fields
title: Text
i didnt feel humiliated
# Question
title: Emotion
description: Provide a single label for the emotion of the text
label_instruction: Select the appropriate label from the list of provided labels.
labels: ['joy', 'anger', 'sadness', 'fear', 'surprise', 'love']

# Distilabel can label your records

We can use the `process` method of the `ArgillaLabeller` class to label the records. This method will label the records with the LLM and update the records with the new labels. We will use this method to label the records of the datasets.

In [31]:
# Process the pending records
result = next(
    labeller.process(
        [
            {
                "record": record,
                "fields": dataset.fields,
                "question": dataset.questions[0],
                "guidelines": dataset.guidelines,
            }
            for record in pending_records
        ]
    )
)
suggestion = result[0]["suggestion"]

print(suggestion)

{'value': 'sadness', 'question_name': 'emotion', 'type': 'model', 'agent': 'llama-3.2-1b-instruct-q8_0.gguf'}

Above, we can see that the labeler has labelled the records with the LLM. We could then take this suggestion and add it back to Argilla to be reviwed by a human.

In [ ]:
record = pending_records[0]
record.suggestions.add(rg.Suggestion(**suggestion["suggestion"]))
dataset.records.log(pending_records)

![label dataset argilla labeller](./images/label_datasets%20with_llms_annotation_guidelines_and_distilabel.png)

# Setup an active loop

In reality, we would want to set up a parallel loop that continuously fetches pending records from Argilla and labels them with the LLM. Below, we can implement this as a simple while loop.

In [ ]:
from time import sleep

while True:

    # query argilla for records that have been responded to
    pending_records = list(
        dataset.records(
            query=rg.Query(filter=rg.Filter(("status", "==", "pending"))),
            limit=1,
        )
    )

    if not pending_records:
        sleep(5)
        continue

    # label the pending records with the LLM based on the dataset settings
    results = next(
        labeller.process(
            [
                {
                    "record": record,
                    "fields": dataset.fields,
                    "question": dataset.questions[0],
                    "guidelines": dataset.guidelines,
                }
                for record in pending_records
            ]
        )
    )

    # add the suggestions to the records
    for record, suggestion in zip(pending_records, results):
        record.suggestions.add(rg.Suggestion(**suggestion["suggestion"]))

    # log the records with the suggestions back to argilla
    dataset.records.log(pending_records)

# 🎉 That's it

That's it! We have successfully labelled the datasets with LLMs based annotation guidelines.